In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
ls

drive/  sample_data/


In [3]:
import os
import glob
import subprocess
import json
import pandas as pd
import torch
import gc
from google.colab import userdata
from openai import OpenAI
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration, Pix2StructProcessor, Pix2StructForConditionalGeneration

# =============================================================================
# AYARLAR VE PATHLER
# =============================================================================
PROJECT_PATH = "/content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST"
PDF_DIR = os.path.join(PROJECT_PATH, "data/retrieved_docs")
INTERMEDIATE_DIR = os.path.join(PROJECT_PATH, "data/processed_markdowns") # Ara çıktı (Markdownlar)
FINAL_DIR = os.path.join(PROJECT_PATH, "data/extraction_results")

os.makedirs(INTERMEDIATE_DIR, exist_ok=True)
os.makedirs(FINAL_DIR, exist_ok=True)

# API Ayarı
try:
    api_key = userdata.get('GROQ_API_KEY')
    client = OpenAI(api_key=api_key, base_url="https://api.groq.com/openai/v1")
except:
    print("⚠️ API Key bulunamadı!")

In [4]:
# =============================================================================
# AŞAMA 1: MARKER İLE TOPLU DÖNÜŞÜM (PDF -> MARKDOWN)
# =============================================================================
def phase_1_marker_batch():
    print("\n🏭 [AŞAMA 1] MARKER ÇALIŞIYOR (PDF -> MARKDOWN)...")

    pdf_files = glob.glob(os.path.join(PDF_DIR, "*.pdf"))
    if not pdf_files:
        print("❌ Hiç PDF bulunamadı.")
        return

    for i, pdf_path in enumerate(pdf_files):
        filename = os.path.basename(pdf_path)
        pdf_slug = os.path.splitext(filename)[0]
        output_subfolder = os.path.join(INTERMEDIATE_DIR, pdf_slug)

        # Eğer daha önce yapıldıysa atla (Resume özelliği)
        if os.path.exists(os.path.join(output_subfolder, f"{pdf_slug}.md")):
            print(f"   ⏩ {filename} zaten işlenmiş, geçiliyor.")
            continue

        print(f"   🔨 İşleniyor ({i+1}/{len(pdf_files)}): {filename}")

        cmd = [
            "marker_single", pdf_path,
            "--output_dir", output_subfolder,
            "--paginate_output", "--force_ocr"
        ]

        try:
            subprocess.run(cmd, check=True, capture_output=True)
        except Exception as e:
            print(f"   ❌ Hata ({filename}): {e}")

    print("✅ Aşama 1 Tamamlandı.")



In [5]:
# =============================================================================
# AŞAMA 2: GÖRSEL ANALİZ (DEPLOT + BLIP)
# =============================================================================
def phase_2_visual_analysis():
    print("\n👁️ [AŞAMA 2] GÖRSEL ANALİZ MODELLERİ YÜKLENİYOR...")

    # Bellek Temizliği (Marker'dan kalanları sil)
    torch.cuda.empty_cache()
    gc.collect()

    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Modelleri Yükle (Sadece bir kez!)
    try:
        blip_proc = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
        blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)

        deplot_proc = Pix2StructProcessor.from_pretrained("google/deplot")
        deplot_model = Pix2StructForConditionalGeneration.from_pretrained("google/deplot").to(device)
        print("✅ Görsel Modeller Bellekte.")
    except Exception as e:
        print(f"❌ Model Yükleme Hatası: {e}")
        return

    # Tüm işlenmiş klasörleri gez
    processed_folders = glob.glob(os.path.join(INTERMEDIATE_DIR, "*"))

    for folder in processed_folders:
        folder_name = os.path.basename(folder)
        visual_report_path = os.path.join(folder, "visual_analysis.txt")

        # Zaten analiz edildiyse atla
        if os.path.exists(visual_report_path):
            continue

        images = glob.glob(os.path.join(folder, "*"))
        visual_summaries = []
        print(f"   🖼️ Analiz Ediliyor: {folder_name}")

        for img_path in images:
            if not img_path.lower().endswith(('.png', '.jpg', '.jpeg')): continue

            try:
                image = Image.open(img_path).convert("RGB")
                if image.width < 150 or image.height < 150: continue # İkonları atla

                # Grafik mi?
                inputs = blip_proc(image, "Is this a chart or a graph?", return_tensors="pt").to(device)
                out = blip_model.generate(**inputs, max_new_tokens=20)
                is_plot = "yes" in blip_proc.decode(out[0], skip_special_tokens=True).lower()

                img_name = os.path.basename(img_path)

                if is_plot:
                    # DePlot
                    d_in = deplot_proc(images=image, text="Generate underlying data table of the figure below:", return_tensors="pt").to(device)
                    d_out = deplot_model.generate(**d_in, max_new_tokens=512)
                    data = deplot_proc.decode(d_out[0], skip_special_tokens=True)
                    visual_summaries.append(f"📊 [CHART DATA - {img_name}]:\n{data}")
                else:
                    # BLIP
                    b_in = blip_proc(image, return_tensors="pt").to(device)
                    b_out = blip_model.generate(**b_in, max_new_tokens=50)
                    desc = blip_proc.decode(b_out[0], skip_special_tokens=True)
                    visual_summaries.append(f"📷 [IMAGE DESC - {img_name}]: {desc}")

            except: continue

        # Sonucu kaydet
        with open(visual_report_path, "w", encoding="utf-8") as f:
            f.write("\n\n".join(visual_summaries))

    # Modelleri silip belleği boşaltalım (LLM için yer kalsın)
    del blip_model, deplot_model
    torch.cuda.empty_cache()
    print("✅ Aşama 2 Tamamlandı.")



In [8]:
# =============================================================================
# AŞAMA 3: LLM REASONING VE SONUÇ ÇIKARMA
# =============================================================================
def phase_3_llm_extraction():
    print("\n🧠 [AŞAMA 3] GROQ (LLAMA 3.3) İLE VERİ ÇIKARMA...")

    processed_folders = glob.glob(os.path.join(INTERMEDIATE_DIR, "*"))
    extraction_results = []

    for folder in processed_folders:
        track_id = os.path.basename(folder)

        # Dosyaları Oku
        md_files = glob.glob(os.path.join(folder, "*.md"))
        visual_file = os.path.join(folder, "visual_analysis.txt")

        if not md_files: continue

        text_content = open(md_files[0], "r", encoding="utf-8").read()
        visual_content = open(visual_file, "r", encoding="utf-8").read() if os.path.exists(visual_file) else "No visual data."

        full_context = f"--- VISUALS ---\n{visual_content}\n\n--- TEXT ---\n{text_content}"

        print(f"   🤖 Groq Analiz Ediyor: {track_id}...")

        # --- PROMPT ---
        PROMPT = """
        Role: IEEE COMST Reviewer.
        Task: Extract structured data from the paper. Use 'NR' for missing info.

        Output JSON:
        {
          "Identity": {"title": "...", "year": "...", "medium": "Fiber/Wireless/Hybrid"},
          "Metrics": {"rate_gbps": (float/NR), "range_m": (float/NR), "accuracy": "..."},
          "Strategic": {"gap": "...", "tier": "1/2/3"}
        }
        """

        try:
            resp = client.chat.completions.create(
                model="llama-3.3-70b-versatile",
                messages=[
                    {"role": "system", "content": PROMPT},
                    {"role": "user", "content": full_context[:90000]} # Limit koruması
                ],
                response_format={"type": "json_object"}
            )
            data = json.loads(resp.choices[0].message.content)

            # Tabloya Ekle
            row = {
                "Track_ID": track_id,
                "Title": data.get("Identity", {}).get("title"),
                "Year": data.get("Identity", {}).get("year"),
                "Medium": data.get("Identity", {}).get("medium"),
                "Rate_Gbps": data.get("Metrics", {}).get("rate_gbps"),
                "Range_m": data.get("Metrics", {}).get("range_m"),
                "Accuracy": data.get("Metrics", {}).get("accuracy"),
                "Gap_Analysis": data.get("Strategic", {}).get("gap"),
                "Tier": data.get("Strategic", {}).get("tier")
            }
            extraction_results.append(row)

        except Exception as e:
            print(f"   ❌ API Hatası ({track_id}): {e}")

    # CSV Kaydet
    if extraction_results:
        df = pd.DataFrame(extraction_results)
        final_csv = os.path.join(FINAL_DIR, "Final_Results.csv")
        df.to_csv(final_csv, index=False)
        print(f"\n🎉 SONUÇLAR KAYDEDİLDİ: {final_csv}")
        print(df.head())



In [ ]:
# =============================================================================
# PİPELINE BAŞLAT
# =============================================================================
phase_1_marker_batch()      # Önce hepsini Markdown yap
phase_2_visual_analysis()   # Sonra hepsinin resimlerini analiz et
phase_3_llm_extraction()    # En son hepsini Groq'a sor

In [7]:
# =============================================================================
# 🔬 TEK DOSYA İÇİN LLM ÖNCESİ PERFORMANS TESTİ
# =============================================================================
import os
import glob
import subprocess
from PIL import Image

def inspect_pre_llm_output():
    # 1. Dosya Seçimi
    pdf_files = glob.glob(os.path.join(PDF_DIR, "*.pdf"))
    if not pdf_files:
        print("⚠️ HATA: Klasörde PDF bulunamadı!")
        return

    target_pdf = pdf_files[0] # İlk dosyayı seç
    filename = os.path.basename(target_pdf)
    pdf_slug = os.path.splitext(filename)[0]

    print(f"🧪 İNCELENEN DOSYA: {filename}")
    print("-" * 60)

    # -------------------------------------------------------------------------
    # AŞAMA 1: MARKER (Metin ve Formül Çıkarma)
    # -------------------------------------------------------------------------
    print("1️⃣ [Marker] Çalışıyor... (PDF -> Markdown + LaTeX)")

    # Geçici test klasörü
    test_output_dir = os.path.join(PROJECT_PATH, "data/test_output", pdf_slug)
    os.makedirs(test_output_dir, exist_ok=True)

    cmd = [
        "marker_single", target_pdf,
        "--output_dir", test_output_dir,
        "--paginate_output", "--force_ocr"
    ]

    try:
        subprocess.run(cmd, check=True, capture_output=True)
        print("   ✅ Marker işlemi tamamlandı.")
    except subprocess.CalledProcessError as e:
        print(f"   ❌ Marker Hatası:\n{e.stderr.decode()}")
        return

    # -------------------------------------------------------------------------
    # AŞAMA 2: GÖRSEL ANALİZ (Grafik -> Tablo / Resim -> Açıklama)
    # -------------------------------------------------------------------------
    print("\n2️⃣ [Görsel Zeka] Resimler Analiz Ediliyor (DePlot + BLIP)...")

    # Marker'ın çıkardığı resimleri bul
    images = glob.glob(os.path.join(test_output_dir, "*"))
    visual_analyses = []

    # Modellerin yüklü olduğundan emin olalım (Global scope'dan)
    if 'blip_model' not in globals() or 'deplot_model' not in globals():
        print("⚠️ UYARI: Modeller bellekte değil. Lütfen önce 'Adım 2' kodunu çalıştırın.")
        return

    for img_path in sorted(images):
        if not img_path.lower().endswith(('.png', '.jpg', '.jpeg')): continue

        try:
            image = Image.open(img_path).convert("RGB")
            if image.width < 100 or image.height < 100: continue

            img_name = os.path.basename(img_path)

            # Grafik mi?
            inputs = blip_processor(image, "Is this a chart or a graph?", return_tensors="pt").to(device)
            out = blip_model.generate(**inputs, max_new_tokens=20)
            is_plot = "yes" in blip_processor.decode(out[0], skip_special_tokens=True).lower()

            result_text = ""
            if is_plot:
                # DePlot ile Tablo Çıkar
                d_in = deplot_processor(images=image, text="Generate underlying data table of the figure below:", return_tensors="pt").to(device)
                d_out = deplot_model.generate(**d_in, max_new_tokens=512)
                data = deplot_processor.decode(d_out[0], skip_special_tokens=True)
                result_text = f"📊 **[GRAFİK VERİSİ - {img_name}]:**\n{data}"
            else:
                # BLIP ile Betimle
                b_in = blip_processor(image, return_tensors="pt").to(device)
                b_out = blip_model.generate(**b_in, max_new_tokens=50)
                desc = blip_processor.decode(b_out[0], skip_special_tokens=True)
                result_text = f"📷 **[RESİM AÇIKLAMASI - {img_name}]:** {desc}"

            visual_analyses.append(result_text)
            print(f"   ok: {img_name} ({'Grafik' if is_plot else 'Resim'})")

        except Exception as e:
            print(f"   hata: {img_name} - {e}")

    # -------------------------------------------------------------------------
    # SONUÇLARI BİRLEŞTİR VE GÖSTER
    # -------------------------------------------------------------------------
    # Metni Oku
    md_file = os.path.join(test_output_dir, f"{pdf_slug}.md")
    if os.path.exists(md_file):
        with open(md_file, "r", encoding="utf-8") as f:
            text_content = f.read()
    else:
        text_content = "⚠️ Markdown dosyası bulunamadı."

    visual_content = "\n\n".join(visual_analyses)

    full_preview = f"""
=====================================================================
DOSYA: {filename}
=====================================================================

[BÖLÜM A] GÖRSEL ANALİZ ÇIKTILARI (LLM BU VERİYİ GÖRECEK)
---------------------------------------------------------------------
{visual_content if visual_content else "Görsel veri yok."}

[BÖLÜM B] MARKER METİN ÇIKTISI (İLK 2000 KARAKTER)
---------------------------------------------------------------------
{text_content[:2000]}
... (Devamı var) ...
    """

    print("\n" + "="*60)
    print(full_preview)
    print("="*60)

    # Dosyaya Kaydet
    save_path = os.path.join(test_output_dir, "PRE_LLM_INSPECTION.txt")
    with open(save_path, "w", encoding="utf-8") as f:
        f.write(full_preview)

    print(f"\n✅ İnceleme dosyası kaydedildi: {save_path}")
    print("👉 Bu çıktıdaki formülleri ($...$) ve grafik tablolarını kontrol edebilirsiniz.")

# Çalıştır
inspect_pre_llm_output()

🧪 İNCELENEN DOSYA: O_ISAC_003.pdf
------------------------------------------------------------
1️⃣ [Marker] Çalışıyor... (PDF -> Markdown + LaTeX)


FileNotFoundError: [Errno 2] No such file or directory: 'marker_single'

In [9]:
import os
import glob
import subprocess
import torch
import gc
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration, Pix2StructProcessor, Pix2StructForConditionalGeneration

def test_pipeline_single_pdf():
    # --- AYARLAR ---
    TARGET_PDF_INDEX = 0  # Klasördeki kaçıncı PDF test edilecek? (0 = İlk dosya)

    # Yollar (Senin pathlerin)
    PROJECT_PATH = "/content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST"
    PDF_DIR = os.path.join(PROJECT_PATH, "data/retrieved_docs")
    TEST_OUTPUT_DIR = os.path.join(PROJECT_PATH, "data/test_output_debug")

    os.makedirs(TEST_OUTPUT_DIR, exist_ok=True)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"⚙️ Donanım: {device.upper()}")

    # 1. DOSYA SEÇİMİ
    pdf_files = glob.glob(os.path.join(PDF_DIR, "*.pdf"))
    if not pdf_files:
        print("❌ Hata: PDF klasörü boş!")
        return

    target_pdf = pdf_files[TARGET_PDF_INDEX]
    filename = os.path.basename(target_pdf)
    slug = os.path.splitext(filename)[0]
    output_subfolder = os.path.join(TEST_OUTPUT_DIR, slug)

    print(f"\n🧪 TEST BAŞLIYOR: {filename}")
    print("="*50)

    # 2. MARKER ÇALIŞTIRMA (OCR & TEXT)
    print("1️⃣ [Marker] Metin ve Resimler Ayrıştırılıyor...")
    cmd = [
        "marker_single", target_pdf,
        "--output_dir", output_subfolder,
        "--paginate_output", "--force_ocr"
    ]

    try:
        subprocess.run(cmd, check=True, capture_output=True)
        print("   ✅ Marker işlemi başarılı.")
    except subprocess.CalledProcessError as e:
        print(f"   ❌ Marker Hatası:\n{e.stderr.decode()}")
        return

    # 3. MODELLERİ YÜKLEME (Sadece test için geçici yükleme)
    print("\n2️⃣ [AI Vision] Modeller Yükleniyor (BLIP + DePlot)...")
    try:
        blip_proc = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
        blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)

        deplot_proc = Pix2StructProcessor.from_pretrained("google/deplot")
        deplot_model = Pix2StructForConditionalGeneration.from_pretrained("google/deplot").to(device)
    except Exception as e:
        print(f"   ❌ Model yükleme hatası: {e}")
        return

    # 4. GÖRSEL ANALİZ
    print("3️⃣ [AI Vision] Görseller Analiz Ediliyor...")
    images = glob.glob(os.path.join(output_subfolder, "*"))
    visual_logs = []

    for img_path in sorted(images):
        if not img_path.lower().endswith(('.png', '.jpg', '.jpeg')): continue

        try:
            image = Image.open(img_path).convert("RGB")
            # Çok küçük ikonları ele
            if image.width < 100 or image.height < 100: continue

            img_name = os.path.basename(img_path)

            # Grafik mi? (Basit bir BLIP sorusu ile)
            inputs = blip_proc(image, "Is this a chart or a graph?", return_tensors="pt").to(device)
            out = blip_model.generate(**inputs, max_new_tokens=20)
            is_plot = "yes" in blip_proc.decode(out[0], skip_special_tokens=True).lower()

            res_text = ""
            if is_plot:
                # DePlot (Grafikten Veriye)
                d_in = deplot_proc(images=image, text="Generate underlying data table of the figure below:", return_tensors="pt").to(device)
                d_out = deplot_model.generate(**d_in, max_new_tokens=512)
                data = deplot_proc.decode(d_out[0], skip_special_tokens=True)
                res_text = f"📊 [GRAFİK VERİSİ - {img_name}]\n{data}"
            else:
                # BLIP (Resim Açıklama)
                b_in = blip_proc(image, return_tensors="pt").to(device)
                b_out = blip_model.generate(**b_in, max_new_tokens=50)
                desc = blip_proc.decode(b_out[0], skip_special_tokens=True)
                res_text = f"📷 [RESİM AÇIKLAMASI - {img_name}]\n{desc}"

            visual_logs.append(res_text)
            print(f"   👉 İşlendi: {img_name} ({'Grafik' if is_plot else 'Resim'})")

        except Exception as e:
            print(f"   ⚠️ Hata ({img_name}): {e}")

    # 5. RAPORLAMA VE ÖNİZLEME
    # Markdown dosyasını oku
    md_path = os.path.join(output_subfolder, f"{slug}.md")
    text_content = open(md_path, "r", encoding="utf-8").read() if os.path.exists(md_path) else "MD Dosyası Yok."

    visual_content = "\n\n".join(visual_logs)

    full_prompt_preview = f"""
    **************************************************
    LLM'E GİDECEK OLAN BİRLEŞTİRİLMİŞ FORMAT
    **************************************************

    --- VISUAL CONTEXT (AI Vision Sonuçları) ---
    {visual_content}

    --- TEXT CONTEXT (Marker Çıktısı - İlk 1500 Karakter) ---
    {text_content[:1500]}
    ...
    """

    print("\n" + "="*50)
    print(full_prompt_preview)
    print("="*50)

    # Raporu kaydet
    report_path = os.path.join(TEST_OUTPUT_DIR, "FINAL_DEBUG_REPORT.txt")
    with open(report_path, "w", encoding="utf-8") as f:
        f.write(full_prompt_preview)

    print(f"\n✅ Rapor Kaydedildi: {report_path}")

    # Bellek Temizliği
    del blip_model, deplot_model, blip_proc, deplot_proc
    torch.cuda.empty_cache()
    gc.collect()

# Çalıştır
test_pipeline_single_pdf()

⚙️ Donanım: CUDA

🧪 TEST BAŞLIYOR: O_ISAC_003.pdf
1️⃣ [Marker] Metin ve Resimler Ayrıştırılıyor...


FileNotFoundError: [Errno 2] No such file or directory: 'marker_single'

In [10]:
# 1. Marker ve gerekli bağımlılıkları yükle
print("📦 Marker Kütüphanesi Yükleniyor... (Bu işlem 1-2 dk sürebilir)")
!pip install marker-pdf --quiet

# 2. Kurulumun başarılı olup olmadığını ve yolunu kontrol et
import shutil

marker_path = shutil.which("marker_single")

if marker_path:
    print(f"\n✅ KURULUM BAŞARILI! Marker yolu: {marker_path}")
    print("👉 Şimdi bir önceki 'test_pipeline_single_pdf' fonksiyonunu tekrar çalıştırabilirsiniz.")
else:
    print("\n❌ HATA: Yükleme yapıldı ama 'marker_single' komutu hala bulunamıyor.")
    print("Lütfen runtime'ı yeniden başlatıp (Runtime > Restart Session) tekrar deneyin.")

📦 Marker Kütüphanesi Yükleniyor... (Bu işlem 1-2 dk sürebilir)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.9/188.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.2/223.2 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 120.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 144.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.4/226.4 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 126.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 63.1

In [11]:
test_pipeline_single_pdf()

⚙️ Donanım: CUDA

🧪 TEST BAŞLIYOR: O_ISAC_003.pdf
1️⃣ [Marker] Metin ve Resimler Ayrıştırılıyor...


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


   ✅ Marker işlemi başarılı.

2️⃣ [AI Vision] Modeller Yükleniyor (BLIP + DePlot)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

3️⃣ [AI Vision] Görseller Analiz Ediliyor...


    **************************************************
    LLM'E GİDECEK OLAN BİRLEŞTİRİLMİŞ FORMAT
    **************************************************
    
    --- VISUAL CONTEXT (AI Vision Sonuçları) ---
    
    
    --- TEXT CONTEXT (Marker Çıktısı - İlk 1500 Karakter) ---
    MD Dosyası Yok.
    ...
    

✅ Rapor Kaydedildi: /content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST/data/test_output_debug/FINAL_DEBUG_REPORT.txt


In [12]:
import os

# Sizin proje yolunuz
PROJECT_PATH = "/content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST"
TEST_OUTPUT_DIR = os.path.join(PROJECT_PATH, "data/test_output_debug")

print(f"📂 Klasör taranıyor: {TEST_OUTPUT_DIR}")
print("-" * 50)

# Klasörde ne var ne yok her şeyi listele
found_any = False
for root, dirs, files in os.walk(TEST_OUTPUT_DIR):
    level = root.replace(TEST_OUTPUT_DIR, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f"{indent}📁 {os.path.basename(root)}/")
    subindent = ' ' * 4 * (level + 1)
    for f in files:
        found_any = True
        print(f"{subindent}📄 {f}")

if not found_any:
    print("\n❌ HATA: Klasör tamamen boş! Marker yazma izni hatası almış olabilir.")
else:
    print("\n✅ Dosya yapısı yukarıdaki gibi. Şimdi dosya ismini görüp düzeltebiliriz.")

📂 Klasör taranıyor: /content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST/data/test_output_debug
--------------------------------------------------
📁 test_output_debug/
    📄 FINAL_DEBUG_REPORT.txt
    📁 O_ISAC_003/
        📁 O_ISAC_003/
            📄 O_ISAC_003.md
            📄 O_ISAC_003_meta.json
            📄 _page_1_Picture_3.jpeg
            📄 _page_1_Picture_4.jpeg
            📄 _page_1_Picture_9.jpeg
            📄 _page_1_Picture_10.jpeg
            📄 _page_2_Figure_15.jpeg
            📄 _page_3_Figure_6.jpeg
            📄 _page_3_Figure_9.jpeg
            📄 _page_4_Figure_11.jpeg

✅ Dosya yapısı yukarıdaki gibi. Şimdi dosya ismini görüp düzeltebiliriz.


In [13]:
import os
import glob
import subprocess
import torch
import gc
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration, Pix2StructProcessor, Pix2StructForConditionalGeneration

def test_pipeline_single_pdf_final_fix():
    # --- AYARLAR ---
    TARGET_PDF_INDEX = 0

    # Yollar (Senin proje yolların)
    PROJECT_PATH = "/content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST"
    PDF_DIR = os.path.join(PROJECT_PATH, "data/retrieved_docs")
    TEST_OUTPUT_DIR = os.path.join(PROJECT_PATH, "data/test_output_debug")

    os.makedirs(TEST_OUTPUT_DIR, exist_ok=True)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"⚙️ Donanım: {device.upper()}")

    # 1. DOSYA SEÇİMİ
    pdf_files = glob.glob(os.path.join(PDF_DIR, "*.pdf"))
    if not pdf_files:
        print("❌ Hata: PDF klasörü boş!")
        return

    target_pdf = pdf_files[TARGET_PDF_INDEX]
    filename = os.path.basename(target_pdf)
    slug = os.path.splitext(filename)[0]

    # Marker çıktı kök klasörü
    output_root = os.path.join(TEST_OUTPUT_DIR, slug)

    print(f"\n🧪 TEST BAŞLIYOR: {filename}")
    print("="*50)

    # 2. MARKER ÇALIŞTIRMA (Akıllı Resume Özelliği ile)
    # Zaten daha önce çalıştıysa tekrar yapma, zaman kazan.
    existing_md = glob.glob(os.path.join(output_root, "**", "*.md"), recursive=True)

    if existing_md:
        print(f"⏩ Marker çıktısı zaten var, atlanıyor... ({os.path.basename(existing_md[0])})")
    else:
        print("1️⃣ [Marker] Metin ve Resimler Ayrıştırılıyor...")
        cmd = [
            "marker_single", target_pdf,
            "--output_dir", output_root,
            "--paginate_output", "--force_ocr"
        ]
        try:
            subprocess.run(cmd, check=True, capture_output=True)
            print("   ✅ Marker işlemi başarılı.")
        except subprocess.CalledProcessError as e:
            print(f"   ❌ Marker Hatası:\n{e.stderr.decode()}")
            return

    # 3. MD VE RESİMLERİ BULMA (Recursive - Can Alıcı Nokta Burası!)
    # Marker nereye kaydettiyse bulup getirecek.
    md_files = glob.glob(os.path.join(output_root, "**", "*.md"), recursive=True)
    if not md_files:
        print("❌ HATA: Marker çalıştı ama .md dosyası bulunamadı!")
        return

    target_md_file = md_files[0]
    target_folder = os.path.dirname(target_md_file) # Resimler bu klasördedir

    print(f"   📂 Çalışma Klasörü Tespit Edildi: {target_folder}")

    # 4. MODELLERİ YÜKLEME
    print("\n2️⃣ [AI Vision] Modeller Yükleniyor (BLIP + DePlot)...")
    try:
        blip_proc = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
        blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)

        deplot_proc = Pix2StructProcessor.from_pretrained("google/deplot")
        deplot_model = Pix2StructForConditionalGeneration.from_pretrained("google/deplot").to(device)
    except Exception as e:
        print(f"   ❌ Model yükleme hatası: {e}")
        return

    # 5. GÖRSEL ANALİZ
    print("3️⃣ [AI Vision] Görseller Analiz Ediliyor...")
    images = glob.glob(os.path.join(target_folder, "*"))
    visual_logs = []

    for img_path in sorted(images):
        if not img_path.lower().endswith(('.png', '.jpg', '.jpeg')): continue

        try:
            image = Image.open(img_path).convert("RGB")
            # Çok küçük ikonları ele (Logo vs.)
            if image.width < 100 or image.height < 100: continue

            img_name = os.path.basename(img_path)

            # Grafik mi?
            inputs = blip_proc(image, "Is this a chart or a graph?", return_tensors="pt").to(device)
            out = blip_model.generate(**inputs, max_new_tokens=20)
            is_plot = "yes" in blip_proc.decode(out[0], skip_special_tokens=True).lower()

            res_text = ""
            if is_plot:
                # DePlot: Grafiği Tabloya Çevir
                d_in = deplot_proc(images=image, text="Generate underlying data table of the figure below:", return_tensors="pt").to(device)
                d_out = deplot_model.generate(**d_in, max_new_tokens=512)
                data = deplot_proc.decode(d_out[0], skip_special_tokens=True)
                res_text = f"📊 [GRAFİK VERİSİ - {img_name}]\n{data}"
            else:
                # BLIP: Resmi Anlat
                b_in = blip_proc(image, return_tensors="pt").to(device)
                b_out = blip_model.generate(**b_in, max_new_tokens=50)
                desc = blip_proc.decode(b_out[0], skip_special_tokens=True)
                res_text = f"📷 [RESİM AÇIKLAMASI - {img_name}]\n{desc}"

            visual_logs.append(res_text)
            print(f"   👉 İşlendi: {img_name} ({'Grafik' if is_plot else 'Resim'})")

        except Exception as e:
            print(f"   ⚠️ Hata ({img_name}): {e}")

    # 6. RAPORLAMA
    text_content = open(target_md_file, "r", encoding="utf-8").read()
    visual_content = "\n\n".join(visual_logs) if visual_logs else "Görsel veri bulunamadı."

    full_prompt_preview = f"""
    **************************************************
    LLM'E GİDECEK OLAN BİRLEŞTİRİLMİŞ FORMAT
    **************************************************

    --- VISUAL CONTEXT (AI Vision Sonuçları) ---
    {visual_content}

    --- TEXT CONTEXT (Marker Çıktısı - İlk 1500 Karakter) ---
    {text_content[:1500]}
    ...
    """

    print("\n" + "="*50)
    print(full_prompt_preview)
    print("="*50)

    report_path = os.path.join(TEST_OUTPUT_DIR, "FINAL_DEBUG_REPORT_FIXED.txt")
    with open(report_path, "w", encoding="utf-8") as f:
        f.write(full_prompt_preview)

    print(f"\n✅ Rapor Kaydedildi: {report_path}")

    # Bellek Temizliği
    del blip_model, deplot_model, blip_proc, deplot_proc
    torch.cuda.empty_cache()
    gc.collect()

# Çalıştır
test_pipeline_single_pdf_final_fix()

⚙️ Donanım: CUDA

🧪 TEST BAŞLIYOR: O_ISAC_003.pdf
⏩ Marker çıktısı zaten var, atlanıyor... (O_ISAC_003.md)
   📂 Çalışma Klasörü Tespit Edildi: /content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST/data/test_output_debug/O_ISAC_003/O_ISAC_003

2️⃣ [AI Vision] Modeller Yükleniyor (BLIP + DePlot)...
3️⃣ [AI Vision] Görseller Analiz Ediliyor...
   👉 İşlendi: _page_1_Picture_10.jpeg (Resim)
   👉 İşlendi: _page_1_Picture_3.jpeg (Resim)
   👉 İşlendi: _page_1_Picture_4.jpeg (Resim)
   👉 İşlendi: _page_1_Picture_9.jpeg (Resim)
   👉 İşlendi: _page_2_Figure_15.jpeg (Resim)
   👉 İşlendi: _page_3_Figure_6.jpeg (Resim)
   👉 İşlendi: _page_3_Figure_9.jpeg (Resim)
   👉 İşlendi: _page_4_Figure_11.jpeg (Resim)


    **************************************************
    LLM'E GİDECEK OLAN BİRLEŞTİRİLMİŞ FORMAT
    **************************************************
    
    --- VISUAL CONTEXT (AI Vision Sonuçları) ---
    📷 [RESİM AÇIKLAMASI - _page_1_Picture_10.jpeg]
a diagram of a car wi

In [14]:
import os
import glob
import subprocess
import torch
import gc
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration, Pix2StructProcessor, Pix2StructForConditionalGeneration

# --- AYARLAR ---
# İşlemek istediğin PDF'in klasördeki sırası (0: İlk dosya, 1: İkinci dosya...)
# Yeni bir PDF denemek istediğin için burayı 1 yapabilirsin.
TARGET_PDF_INDEX = 1

PROJECT_PATH = "/content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST"
PDF_DIR = os.path.join(PROJECT_PATH, "data/retrieved_docs")
TEST_OUTPUT_DIR = os.path.join(PROJECT_PATH, "data/test_output_debug")

# -----------------------------------------------------------------------------
# FONKSİYON: TEK BİR PDF'İ UÇTAN UCA HAZIRLA
# -----------------------------------------------------------------------------
def run_clean_single_pdf_test(target_index):
    os.makedirs(TEST_OUTPUT_DIR, exist_ok=True)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"⚙️ Donanım: {device.upper()}")

    # 1. DOSYA SEÇİMİ
    pdf_files = sorted(glob.glob(os.path.join(PDF_DIR, "*.pdf")))
    if not pdf_files:
        print("❌ Hata: PDF klasörü boş!")
        return

    if target_index >= len(pdf_files):
        print(f"❌ Hata: İndeks {target_index} geçersiz. Toplam {len(pdf_files)} dosya var.")
        return

    target_pdf = pdf_files[target_index]
    filename = os.path.basename(target_pdf)
    slug = os.path.splitext(filename)[0]
    output_root = os.path.join(TEST_OUTPUT_DIR, slug)

    print(f"\n🧪 TEST BAŞLIYOR: {filename}")
    print("="*60)

    # 2. MARKER ÇALIŞTIRMA (Zaten yapıldıysa atla)
    # Recursive arama ile md dosyasını kontrol et
    existing_md = glob.glob(os.path.join(output_root, "**", "*.md"), recursive=True)

    if existing_md:
        print(f"⏩ Marker daha önce çalışmış, atlanıyor... ({os.path.basename(existing_md[0])})")
        target_md_file = existing_md[0]
    else:
        print("1️⃣ [Marker] Metin ve Resimler Ayrıştırılıyor...")
        cmd = [
            "marker_single", target_pdf,
            "--output_dir", output_root,
            "--paginate_output", "--force_ocr"
        ]
        try:
            subprocess.run(cmd, check=True, capture_output=True)
            print("   ✅ Marker işlemi başarılı.")
            # İşlem bitince dosyayı bul
            md_files = glob.glob(os.path.join(output_root, "**", "*.md"), recursive=True)
            if not md_files:
                print("❌ KRİTİK HATA: Marker çalıştı ama çıktı dosyası bulunamadı.")
                return
            target_md_file = md_files[0]
        except subprocess.CalledProcessError as e:
            print(f"   ❌ Marker Hatası:\n{e.stderr.decode()}")
            return

    # Resimlerin olduğu klasörü tespit et (MD dosyasının yanındadır)
    target_folder = os.path.dirname(target_md_file)
    print(f"   📂 Çalışma Klasörü: {target_folder}")

    # 3. MODELLERİ YÜKLEME
    print("\n2️⃣ [AI Vision] Modeller Yükleniyor (BLIP + DePlot)...")
    try:
        blip_proc = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
        blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)

        deplot_proc = Pix2StructProcessor.from_pretrained("google/deplot")
        deplot_model = Pix2StructForConditionalGeneration.from_pretrained("google/deplot").to(device)
    except Exception as e:
        print(f"   ❌ Model yükleme hatası: {e}")
        return

    # 4. GÖRSEL ANALİZ (GÜÇLENDİRİLMİŞ MANTIK)
    print("3️⃣ [AI Vision] Görseller Analiz Ediliyor (Akıllı Algılama Modu)...")
    images = glob.glob(os.path.join(target_folder, "*"))
    visual_logs = []

    for img_path in sorted(images):
        if not img_path.lower().endswith(('.png', '.jpg', '.jpeg')): continue

        try:
            image = Image.open(img_path).convert("RGB")
            if image.width < 100 or image.height < 100: continue

            img_name = os.path.basename(img_path)

            # --- STRATEJİ: Grafik mi Resim mi? ---
            use_deplot = False

            # Kural 1: Dosya isminde "Figure", "Chart", "Plot" varsa şüphelen
            if any(x in img_name.lower() for x in ["figure", "chart", "plot", "fig"]):
                use_deplot = True # Şimdilik aday

            # Kural 2: BLIP'e sor (Caption al)
            inputs = blip_proc(image, return_tensors="pt").to(device)
            out = blip_model.generate(**inputs, max_new_tokens=50)
            desc = blip_proc.decode(out[0], skip_special_tokens=True).lower()

            # Kural 3: Açıklamada grafik terimleri geçiyor mu?
            chart_keywords = ["graph", "plot", "chart", "diagram", "axis", "curves"]
            if any(k in desc for k in chart_keywords):
                use_deplot = True

            log_text = ""
            if use_deplot:
                # DePlot ile VERİ ÇEKMEYİ DENE
                try:
                    d_in = deplot_proc(images=image, text="Generate underlying data table of the figure below:", return_tensors="pt").to(device)
                    d_out = deplot_model.generate(**d_in, max_new_tokens=512)
                    data_table = deplot_proc.decode(d_out[0], skip_special_tokens=True)

                    # Eğer DePlot saçmaladıysa (çok kısa çıktı), BLIP sonucuna dön
                    if len(data_table) < 10:
                        log_text = f"📷 [RESİM - {img_name}]\n{desc} (DePlot veri çıkaramadı)"
                        print(f"   ⚠️ {img_name}: Grafik sanıldı ama veri çıkmadı. Resim moduna dönüldü.")
                    else:
                        log_text = f"📊 [GRAFİK VERİSİ - {img_name}]\n**Context:** {desc}\n**Data Table:**\n{data_table}"
                        print(f"   📊 {img_name}: Tabloya dönüştürüldü.")
                except:
                    # DePlot hata verirse BLIP'e dön
                    log_text = f"📷 [RESİM - {img_name}]\n{desc}"
            else:
                # Sadece Resim
                log_text = f"📷 [RESİM - {img_name}]\n{desc}"
                print(f"   📷 {img_name}: Resim olarak işlendi.")

            visual_logs.append(log_text)

        except Exception as e:
            print(f"   ⚠️ Hata ({img_name}): {e}")

    # 5. RAPORLAMA
    text_content = open(target_md_file, "r", encoding="utf-8").read()
    visual_content = "\n\n".join(visual_logs) if visual_logs else "Görsel veri bulunamadı."

    full_prompt_preview = f"""
==================================================
LLM HAZIRLIK RAPORU: {filename}
==================================================

[BÖLÜM 1] GÖRSEL ANALİZ (Vision AI)
--------------------------------------------------
{visual_content}

[BÖLÜM 2] METİN İÇERİĞİ (Marker - İlk 2000 Karakter)
--------------------------------------------------
{text_content[:2000]}
...
    """

    # Raporu kaydet
    report_filename = f"PREVIEW_{slug}.txt"
    report_path = os.path.join(TEST_OUTPUT_DIR, report_filename)
    with open(report_path, "w", encoding="utf-8") as f:
        f.write(full_prompt_preview)

    print("\n" + "="*60)
    print(f"✅ İNCELEME DOSYASI HAZIR: {report_path}")
    print("="*60)
    print(f"👉 Çıktıda '📊 [GRAFİK VERİSİ]' başlıklarını ve altındaki tabloları kontrol et.")

    # Bellek Temizliği
    del blip_model, deplot_model, blip_proc, deplot_proc
    torch.cuda.empty_cache()
    gc.collect()

# --- ÇALIŞTIR ---
# 2. PDF dosyasını denemek için index'i 1 yap (veya istediğin sayı)
run_clean_single_pdf_test(TARGET_PDF_INDEX)

⚙️ Donanım: CUDA

🧪 TEST BAŞLIYOR: O_ISAC_002.pdf
1️⃣ [Marker] Metin ve Resimler Ayrıştırılıyor...
   ✅ Marker işlemi başarılı.
   📂 Çalışma Klasörü: /content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST/data/test_output_debug/O_ISAC_002/O_ISAC_002

2️⃣ [AI Vision] Modeller Yükleniyor (BLIP + DePlot)...
3️⃣ [AI Vision] Görseller Analiz Ediliyor (Akıllı Algılama Modu)...


Arial.TTF: 0.00B [00:00, ?B/s]

   📊 _page_1_Figure_2.jpeg: Tabloya dönüştürüldü.
   📊 _page_1_Figure_6.jpeg: Tabloya dönüştürüldü.
   📊 _page_2_Figure_3.jpeg: Tabloya dönüştürüldü.

✅ İNCELEME DOSYASI HAZIR: /content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST/data/test_output_debug/PREVIEW_O_ISAC_002.txt
👉 Çıktıda '📊 [GRAFİK VERİSİ]' başlıklarını ve altındaki tabloları kontrol et.


In [ ]:
import os
import glob
import subprocess
import torch
import gc
import time
from collections import defaultdict
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration, Pix2StructProcessor, Pix2StructForConditionalGeneration

# =============================================================================
# 1. AYARLAR & KONFİGÜRASYON
# =============================================================================
PROJECT_PATH = "/content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST"
PDF_DIR = os.path.join(PROJECT_PATH, "data/retrieved_docs")
INTERMEDIATE_DIR = os.path.join(PROJECT_PATH, "data/processed_markdowns")

# --- DONANIM AYARLARI ---
# A100/L4 kullanıyorsanız Batch Size'ı artırın (16-32), T4 ise (4-8)
device = "cuda" if torch.cuda.is_available() else "cpu"
VRAM = torch.cuda.get_device_properties(0).total_memory / 1e9 if torch.cuda.is_available() else 0

if VRAM > 30: # A100 (40GB/80GB)
    VISION_BATCH_SIZE = 24
    MARKER_WORKERS = 10
elif VRAM > 20: # L4 (24GB)
    VISION_BATCH_SIZE = 16
    MARKER_WORKERS = 8
else: # T4 (16GB)
    VISION_BATCH_SIZE = 4
    MARKER_WORKERS = 4

print(f"🚀 Donanım: {device.upper()} | VRAM: {VRAM:.1f} GB")
print(f"⚙️ Ayarlar: Vision Batch={VISION_BATCH_SIZE} | Marker Workers={MARKER_WORKERS}")

os.makedirs(INTERMEDIATE_DIR, exist_ok=True)

# =============================================================================
# 2. FONKSİYON: PARALEL MARKER (Tüm PDF'ler Tek Komutla)
# =============================================================================
def phase_1_parallel_marker():
    print(f"\n🏭 [AŞAMA 1] PARALEL MARKER BAŞLATILIYOR...")
    start_time = time.time()

    # Marker'ın "folder mode"unu kullanıyoruz. Bu mod otomatik olarak worker açar.
    # --force_ocr yerine --paginate_output kullanıyoruz, gerekirse OCR'ı marker kendisi tetikler.
    cmd = [
        "marker",
        PDF_DIR,
        "--output_dir", INTERMEDIATE_DIR,
        "--workers", str(MARKER_WORKERS),
        "--paginate_output"
    ]

    try:
        # Check=True hata olursa durdurur
        subprocess.run(cmd, check=True)
        duration = time.time() - start_time
        print(f"✅ Marker Tamamlandı! Süre: {duration:.2f} saniye")
    except subprocess.CalledProcessError as e:
        print(f"❌ Marker Hatası: {e}")

# =============================================================================
# 3. FONKSİYON: BATCH VISION (Görsel İşleme Fabrikası)
# =============================================================================
def phase_2_batch_vision():
    print(f"\n👁️ [AŞAMA 2] BATCH VISION ANALİZİ BAŞLIYOR...")

    # A. Modelleri Yükle
    print("   ⏳ Modeller VRAM'e yükleniyor (BLIP + DePlot)...")
    try:
        blip_proc = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
        blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)

        deplot_proc = Pix2StructProcessor.from_pretrained("google/deplot")
        deplot_model = Pix2StructForConditionalGeneration.from_pretrained("google/deplot").to(device)
    except Exception as e:
        print(f"❌ Model Yükleme Hatası: {e}")
        return

    # B. Tüm Resimleri Topla (Recursive)
    all_images = glob.glob(os.path.join(INTERMEDIATE_DIR, "**", "*.png"), recursive=True)
    all_images += glob.glob(os.path.join(INTERMEDIATE_DIR, "**", "*.jpg"), recursive=True)
    all_images += glob.glob(os.path.join(INTERMEDIATE_DIR, "**", "*.jpeg"), recursive=True)

    # Küçük ikonları ve zaten işlenmiş klasörleri filtrele
    process_queue = []
    processed_folders_cache = set()

    for img_path in all_images:
        parent_folder = os.path.dirname(os.path.dirname(img_path)) # .../slug/slug/img.png -> .../slug

        # Eğer bu PDF için analiz dosyası zaten varsa atla (Resume özelliği)
        if parent_folder in processed_folders_cache: continue
        if os.path.exists(os.path.join(parent_folder, "visual_analysis.txt")):
            processed_folders_cache.add(parent_folder)
            continue

        if os.path.getsize(img_path) > 5000: # 5KB altı muhtemelen logo/ikon
            process_queue.append(img_path)

    total_imgs = len(process_queue)
    if total_imgs == 0:
        print("   ⏩ İşlenecek yeni resim bulunamadı.")
        return

    print(f"   🖼️ Toplam {total_imgs} resim işlenecek. (Tahmini Batch Sayısı: {total_imgs // VISION_BATCH_SIZE})")

    # C. Batch Processing Döngüsü
    results_map = {} # {image_path: analysis_text}

    for i in range(0, total_imgs, VISION_BATCH_SIZE):
        batch_paths = process_queue[i : i + VISION_BATCH_SIZE]
        batch_pil = []
        valid_batch_paths = []

        # 1. Resimleri Yükle
        for p in batch_paths:
            try:
                img = Image.open(p).convert("RGB")
                batch_pil.append(img)
                valid_batch_paths.append(p)
            except: pass

        if not batch_pil: continue

        # 2. Hepsine BLIP (Caption) Uygula - Çok Hızlı
        try:
            inputs = blip_proc(batch_pil, return_tensors="pt", padding=True).to(device)
            out = blip_model.generate(**inputs, max_new_tokens=50)
            captions = blip_proc.batch_decode(out, skip_special_tokens=True)
        except Exception as e:
            print(f"   ⚠️ BLIP Batch Hatası: {e}")
            continue

        # 3. Grafik Olanları Tespit Et ve DePlot'a Hazırla
        deplot_indices = []
        deplot_images = []

        for idx, (cap, path) in enumerate(zip(captions, valid_batch_paths)):
            filename = os.path.basename(path).lower()
            # Strateji: Dosya adında 'figure' varsa VEYA caption 'graph/chart' diyorsa
            is_chart = any(k in filename for k in ['figure', 'fig', 'chart', 'plot']) or \
                       any(k in cap.lower() for k in ['graph', 'chart', 'plot', 'diagram'])

            if is_chart:
                deplot_indices.append(idx)
                deplot_images.append(batch_pil[idx])
            else:
                # Grafik değilse sadece caption kaydet
                results_map[path] = f"📷 [IMAGE]: {cap}"

        # 4. Sadece Grafikleri DePlot'a Sok (Ağır İşlem)
        if deplot_images:
            try:
                d_in = deplot_proc(images=deplot_images, text=["Generate underlying data table of the figure below:"]*len(deplot_images), return_tensors="pt", padding=True).to(device)
                d_out = deplot_model.generate(**d_in, max_new_tokens=512) # Tablo uzun olabilir
                tables = deplot_proc.batch_decode(d_out, skip_special_tokens=True)

                for dp_idx, table_text in zip(deplot_indices, tables):
                    orig_path = valid_batch_paths[dp_idx]
                    orig_cap = captions[dp_idx]

                    # DePlot bazen saçmalar, çok kısaysa BLIP'e dön
                    if len(table_text) < 15:
                        results_map[orig_path] = f"📷 [IMAGE]: {orig_cap}"
                    else:
                        results_map[orig_path] = f"📊 [CHART DATA]\nContext: {orig_cap}\nData:\n{table_text}"
            except Exception as e:
                print(f"   ⚠️ DePlot Batch Hatası: {e}")
                # Kurtarma: Hata verenleri BLIP sonucuyla kaydet
                for dp_idx in deplot_indices:
                    results_map[valid_batch_paths[dp_idx]] = f"📷 [IMAGE]: {captions[dp_idx]} (DePlot Failed)"

        # İlerleme Çubuğu (Log)
        if (i // VISION_BATCH_SIZE) % 5 == 0:
            print(f"   🚀 İlerleme: {i}/{total_imgs} resim tamamlandı.")

    # D. Sonuçları Dosyalara Yaz (Grouping)
    print("   💾 Sonuçlar diske yazılıyor...")
    folder_groups = defaultdict(list)

    for path, text in results_map.items():
        # Dosya yolu: .../slug/slug/image.png
        # Biz visual_analysis.txt'yi .../slug/ (Markdown'ın olduğu yer) altına koymak istiyoruz.
        # Marker yapısı: output_dir/slug/slug/image.png  (İç içe klasör yapıyor)
        # Markdown dosyası: output_dir/slug/slug/slug.md

        grandparent_folder = os.path.dirname(os.path.dirname(path)) # Bu genellikle ana slug klasörü
        folder_groups[grandparent_folder].append(f"--- {os.path.basename(path)} ---\n{text}")

    count_saved = 0
    for folder, logs in folder_groups.items():
        # En emin olduğumuz yöntem: md dosyasını bul, yanına koy.
        md_files = glob.glob(os.path.join(folder, "**", "*.md"), recursive=True)
        if md_files:
            target_dir = os.path.dirname(md_files[0]) # MD dosyasının yanı
            save_path = os.path.join(target_dir, "visual_analysis.txt")
            with open(save_path, "w", encoding="utf-8") as f:
                f.write("\n\n".join(logs))
            count_saved += 1

    print(f"✅ İşlem Bitti! {count_saved} adet PDF için görsel analiz raporu oluşturuldu.")

    # Bellek Temizliği
    del blip_model, deplot_model, blip_proc, deplot_proc
    torch.cuda.empty_cache()
    gc.collect()

# =============================================================================
# 4. KONTROL BLOĞU (Rastgele Bir Sonucu Göster)
# =============================================================================
def inspect_results():
    print("\n🔍 [AŞAMA 3] RASTGELE BİR SONUÇ KONTROL EDİLİYOR...")
    vis_files = glob.glob(os.path.join(INTERMEDIATE_DIR, "**", "visual_analysis.txt"), recursive=True)

    if not vis_files:
        print("❌ Hiç sonuç dosyası bulunamadı.")
        return

    target = vis_files[0]
    print(f"📄 Dosya: {target}")
    print("-" * 50)
    with open(target, "r") as f:
        print(f.read()[:2000]) # İlk 2000 karakter
    print("-" * 50)

# =============================================================================
# BAŞLAT
# =============================================================================
if __name__ == "__main__":
    # 1. Aşama: Tüm PDF'leri çevir
    phase_1_parallel_marker()

    # 2. Aşama: Tüm Resimleri işle
    phase_2_batch_vision()

    # 3. Aşama: Kontrol et
    inspect_results()